# Test analysis

Statistical analysis for 10 repetitions of a 5 minute test and for 10 repetitions of a 1 minute test of the **Sockshop** application

### Workloads

- **5 minute test**:
    - simulated day length: 5 minutes = 300 seconds
    - 20 requests per second
    - about 6000 total requests

- **1 minute test** (compressed test):
    - simulated day length: 1 minutes = 60 seconds
    - 100 requests per second
    - about 6000 total requests

### Imports and functions

In [ ]:
import os
import pathlib
import sys

import numpy as np
import pandas as pd
from IPython.display import HTML, display, display_html

mod_path = os.path.abspath(os.path.join("../../src/alyslib"))
if mod_path not in sys.path:
    sys.path.append(mod_path)

import alyslib

In [ ]:
# function that returns a list that contains
# the mean of the TimeDeltas for every dataframe
def get_means(df):
    l = []
    for d in df:
        l.append(d.TimeDelta.mean())
    return l


# function that calculates the confidence interval
# (with `z_score`=1.96, returns a 95% confidence interval)
def conf_interval(data, z_score=1.96):
    mean = np.mean(data)
    std = np.std(data)
    size = len(data)
    err = z_score * (std / np.sqrt(size))
    return (mean - err, mean + err)

### Datasets - import

In [ ]:
l = alyslib.import_data("./data", "net.gen")

### DataFrames - building

In [ ]:
d0, d1 = alyslib.build_dfs(l)

In [ ]:
dfmerge = d0 + d1

### DataFrames - cleaning network noise

For the analysis of the tests we cannot have **SendIP** and **RecvIP** differences. We clean the network noise for every pair of tests.

In [ ]:
alyslib.clean_network_noise(dfmerge)

### Dataframes - sorting by Timestamp

In [ ]:
alyslib.sort_by_key(dfmerge, "Timestamp")

### Dataframes - generating column Elapsed time

In [ ]:
alyslib.cmp_elapsed(dfmerge)

### DataFrames - removing TimeDelta outliers

We treat as outliers every TimeDelta that is greater than the third TimeDelta quantile (q3 = 0.75).

In [ ]:
alyslib.clean_timedelta_outliers(dfmerge)

### Analysis - 5 minutes test

In [ ]:
d0m = get_means(d0)

In [ ]:
display(pd.DataFrame(d0m, columns=["means"]))
display(
    pd.DataFrame(
        [[np.mean(d0m), np.std(d0m), conf_interval(d0m)]],
        columns=["mean of means", "std of means", "95% conf interval"],
    )
)

1. we calculated the **TimeDelta mean** for every DataFrame generated for the current test (we have 10 repetitions, so we have 10 DataFrames).
2. we calculated the **mean of the means** calculated above
3. we calculated the **standard deviation of the means** calculated above
4. we calculated the **95% confidence interval of the means** calculated above

### Analysis - 1 minute test

In [ ]:
d1m = get_means(d1)

In [ ]:
display(pd.DataFrame(d1m, columns=["means"]))
display(
    pd.DataFrame(
        [[np.mean(d1m), np.std(d1m), conf_interval(d1m)]],
        columns=["mean of means", "std of means", "95% conf interval"],
    )
)

We performed the same calculations as the previous test with this new test (again we have 10 repetitions, so we have 10 DataFrames for the current test)

### Analysis - comparison between the tests

In [ ]:
display(
    pd.DataFrame(
        [
            [np.mean(d0m), np.std(d0m), conf_interval(d0m)],
            [np.mean(d1m), np.std(d1m), conf_interval(d1m)],
        ],
        columns=["mean of means", "std of means", "95% conf interval"],
        index=["5m test", "1m test"],
    )
)

In conclusion, we present above the achieved results to compare the results of both tests simultaneously